# Playing with PyPI (data)

Many thanks to Seth M. Larson for making these available: https://github.com/sethmlarson/pypi-data

In [ ]:
import ibis
from ibis import _

ibis.options.interactive = True

In [ ]:
con = ibis.duckdb.connect()

We earlier downloaded some PyPI maintainer data as parquet files, now let's load
them into DuckDB:

In [ ]:
from pathlib import Path

filenames = [
    "deps.parquet",
    "maintainers.parquet",
    "package_urls.parquet",
    "packages.parquet",
    "scorecard_checks.parquet",
    "wheels.parquet",
]

folder = Path("pypi")

for filename in filenames:
    path = folder / filename
    con.read_parquet(path, table_name=filename.split(".")[0])

There should be several tables available -- we'll be focusing on using the
`deps`, `maintainers`, and `packages` tables:


In [ ]:
con.list_tables()

In [ ]:
deps = con.tables.deps

deps

In [ ]:
maintainers = con.tables.maintainers

maintainers

In [ ]:
packages = con.tables.packages

packages

## Looking for typo squatters

We've all done it, you typo in your `pip install` line and you end up with...
something else. Most of the time, these are honest mistakes.  There are a LOT of
packages.

Sometimes, it is... less honest.

There is a `levenshtein` method for calculating string similarity against a string column.
Let's try that out and see if there are any packages similarly named to, say, `pandas`:

In [ ]:
packages.select("name", sim=packages.name.levenshtein("pandas")).order_by(_.sim)

Those seem like they are probably legitimate projects -- maybe we can try out another text similarity search?

There's a `jaccard` search function in DuckDB, but there's no `jaccard` method on Ibis' string columns...

What's an enterprising Ibis user to do?

## UDF builtins

Modern SQL engines have a _lot_ of features and while Ibis provides a consistent
interface, it can't always expose every option of _every_ backend.

One easy way to enable extra backend functionality is with a udf builtin!

If a function exists on a particular backend (this is backend-specific by
definition), you can create a dummy function that maps the name, the inputs, and
the output type for Ibis to use.

The Jaccard function on DuckDB takes in two strings and returns the similarity
measure as a number between 0 and 1. To enable this in Ibis, we need to make use
of a handy decorator, make sure the function name matches, and add a return type
hint (this is required).

In [ ]:
@ibis.udf.scalar.builtin
def jaccard(left: str, right: str) -> float:
    ...

Now that that's defined, we can use our newly exposed `jaccard` function -- not
as a method on the `name` column, but by passing the column as one of the
arguments:

In [ ]:
packages.select("name", sim=jaccard(packages.name, "pandas")).order_by(_.sim.desc())

### Exercise 1

Try swapping out `jaccard` for a different similarity function available in
DuckDB (https://duckdb.org/docs/sql/functions/char#text-similarity-functions).

### Solution

One possible solution:

In [ ]:
%load solutions/nb04_ex01_jaro.py

## Exploring the PyPI maintainers dataset further

### Exercise 2: Which maintainers have the most downloads?

Which package maintainers on PyPI have the most downloads across all the
packages they maintain?


### Solution

In [ ]:
%load solutions/nb04_ex02.py

### Exercise 3: What packages depend on `ibis-framework`?

Or fill in your package name of choice

(You can use the `distinct` method to return a table with duplicate entries removed.)


### Solution

In [ ]:
%load solutions/nb04_ex03.py

## Using `.sql` as an escape hatch

Ibis is pretty good about abstracting away the underlying execution engine, but
sometimes you _want_ to talk directly to the engine.

Maybe you:
- have an existing SQL query from a data engineer that you want to build on
- need to access a backend-specific function that isn't exposed in Ibis
- really want to write some SQL

If you do, then you can call `.sql` on a connection (or an existing expression!)
and write out your SQL query.

Let's try out a few straightforward examples:

In [ ]:
con.sql("SELECT 1 + 1")

This isn't a necessarily a _good_ way to do math, but it's available!

In [ ]:
con.sql("SELECT 1 + 1 as f")

Now that the column of our simple addition has a name, we can chain on the
expression, the same as any other Ibis expression.

In [ ]:
con.sql("SELECT 1 + 1 as f").mutate(f=_.f + 2)

You can (should you so desire) continue to hop back-and-forth from Ibis land to SQL land:

In [ ]:
con.sql("SELECT 1 + 1 as f").mutate(f=_.f + 2).alias("t").sql("SELECT f, f^2 AS fsquared FROM t")

That... doesn't look right. What happened?

Postgres uses the caret symbol (`^`) for `pow`, but DuckDB doesn't. But we don't
have to rewrite it, just let Ibis know that it's written in a different dialect:

In [ ]:
con.sql("SELECT 1 + 1 as f").mutate(f=_.f + 2).alias("t").sql("SELECT f, f^2 AS fsquared FROM t", dialect="postgres")

### Exercise 4: Find the top 20 most transitively depended on packages

#### Are there limits on what what you can pass to `.sql`?

There are, but they are generous limits. Call to `.sql` are also deferred so we
need to know the `schema` of the result. This allows you to continue chaining
methods after a `.sql` call without executing the full query.

Because of this, the results of `.sql` need to be a table.

But you can do some pretty interesting things with this -- here's a _recursive
common table expression_ that looks up transitive dependencies.

In [ ]:
expr = con.sql("""WITH RECURSIVE
direct_deps(package, dependency) AS (
  SELECT
    package_name,
    dep_name
  FROM deps
  WHERE
    extra IS NULL
),
transitive_deps(package, intermediate, dependency) AS (
  SELECT
    package,
    package,
    dependency
  FROM direct_deps
  UNION
  SELECT
    transitive_deps.package,
    direct_deps.package,
    direct_deps.dependency
  FROM direct_deps
  JOIN transitive_deps
    ON direct_deps.package = transitive_deps.dependency
)
SELECT
*
FROM transitive_deps
""")
expr

Now, given this recursive CTE, can we determine the top 20 most transitively
depended on packages?

### Solution

In [ ]:
%load solutions/nb04_ex04.py

## Additional exercises

### Histogram of maintainer count

Start by computing the number of packages that have `n` maintainers, as
defined by having write permissions on PyPI.

In [ ]:
maintainer_counts = (
    maintainers.group_by("package_name")
    .agg(maintainers=_.count())
    .group_by("maintainers")
    .agg(count=_.count())
    .order_by(_.maintainers)
)

maintainer_counts

We'll plot a histogram of the maintainer counts using `altair`.

In [ ]:
import altair as alt

In [ ]:
chart = (
    alt.Chart(maintainer_counts)
    .mark_bar()
    .encode(x="maintainers", y=alt.Y("count", scale=alt.Scale(type="log")))
)

chart

### Exercise 05

What's the deal with that spike at 12 maintainers?

**Note**: if you would like to return more rows in interactive mode, you can set
  the row-count e.g. `ibis.options.repr.interactive.max_rows = 20`


### Solution

In [ ]:
# %load solutions/nb04_ex05_counts.py

In [ ]:
# %load solutions/nb04_ex05_ftw.py

In [ ]:
# %load solutions/nb04_ex05_ftw_maintainers.py